In [13]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle
from nltk.corpus import stopwords  # remove words like for the with in of
from nltk.stem.porter import PorterStemmer # loving loved==love convert into base word
from sklearn.feature_extraction.text import TfidfVectorizer # Loved=[0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
news_df = pd.read_csv('/content/train.csv', encoding='latin1')
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
news_df.shape

(20800, 5)

In [16]:
news_df.isna().sum()   # it tells how many values are NaN in id,title,authro,text,label

,0
id,0
title,558
author,1957
text,39
label,0


In [17]:
news_df=news_df.fillna(' ')  # it fill all the NaN values with ' '
news_df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [18]:
news_df['content']=news_df['author']+news_df['title']
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell LucusHouse Dem Aide: We Didnât Even ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a âPoster Child For White...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome HudsonRapper T.I.: Trump a âPoster Ch...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat..."
20797,20797,Macyâs Is Said to Receive Takeover Approach ...,Michael J. de la Merced and Rachel Abrams,The Macyâs of today grew from the union of s...,0,Michael J. de la Merced and Rachel AbramsMacyâ...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex AnsaryNATO, Russia To Hold Parallel Exerc..."


In [19]:
news_df['content'][20796]

'Benjamin HoffmanN.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [20]:
# stemming
ps=PorterStemmer()



def stemming(content):
    # Handle the case where content is None or not a string
    if content is None:
        content = ""  # Assign an empty string if content is None
    elif not isinstance(content, str):
        content = str(content)  # Convert to string if it's not a string

    # Replace non-alphabet characters with space
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)

    # Convert to lowercase
    stemmed_content = stemmed_content.lower()

    # Split the string into words
    stemmed_content = stemmed_content.split()

    # Remove stopwords and apply stemming
    stemmed_content = [ps.stem(word) for word in stemmed_content if word not in stopwords.words('english')]

    # Join the stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [21]:
news_df['content'] = news_df['content'].apply(stemming)

In [22]:
news_df['content']

,content
0,darrel lucushous dem aid even see comey letter...
1,daniel j flynnflynn hillari clinton big woman ...
2,consortiumnew comwhi truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoyiranian woman jail fiction unpub...
...,...
20795,jerom hudsonrapp trump poster child white supr...
20796,benjamin hoffmann f l playoff schedul matchup ...
20797,michael j de la merc rachel abramsmaci said re...
20798,alex ansarynato russia hold parallel exercis b...


In [23]:
X=news_df['content'].values
Y=news_df['label'].values
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [24]:
vector=TfidfVectorizer()
vector.fit(X)
X=vector.transform(X)
print(X)

  (0, 578)	0.2694167078545384
  (0, 4211)	0.36253203231506576
  (0, 5006)	0.24725958235728157
  (0, 5969)	0.35488202138141456
  (0, 6273)	0.2839932825877812
  (0, 8022)	0.2313366174248873
  (0, 12782)	0.24619727512767192
  (0, 14555)	0.2917725968420029
  (0, 15019)	0.4300622675963931
  (0, 22724)	0.25523360180691607
  (0, 26340)	0.2808837940159642
  (1, 2622)	0.3562953366945267
  (1, 3281)	0.18652439327549428
  (1, 3859)	0.45980466668763476
  (1, 4767)	0.23338756776626793
  (1, 5916)	0.31810058109638056
  (1, 8772)	0.5258635625386451
  (1, 11313)	0.24166773097712638
  (1, 27923)	0.36911845953845024
  (2, 5121)	0.5511414848555652
  (2, 5240)	0.40440534260277944
  (2, 8567)	0.3411947414020896
  (2, 9454)	0.30743020569262086
  (2, 16361)	0.43295215406038445
  (2, 26235)	0.3665032495181434
  :	:
  (20797, 1249)	0.3072223353708335
  (20797, 2257)	0.3357782642976524
  (20797, 6088)	0.21253094503918346
  (20797, 11692)	0.2992170910232368
  (20797, 14104)	0.22761807337911874
  (20797, 16217)	0

In [25]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)
X_train.shape

(16640, 28419)

In [26]:
model=LogisticRegression()
model.fit(X_train,Y_train)

with open('fakenewsdetection.pkl','wb') as file:
    pickle.dump('model',file)


In [27]:
train_y_pred=model.predict(X_train)
train_accuracy=accuracy_score(train_y_pred,Y_train)
print(train_accuracy)

0.971875


In [28]:
test_y_pred=model.predict(X_test)
test_accuracy=accuracy_score(test_y_pred,Y_test)
print(test_accuracy)

0.9550480769230769


In [29]:
# prediction
x_new=X_test[2]
prediction=model.predict(x_new)
if prediction[0]==0:
  print('The news is real')
else:
  print('The news is fake')

The news is fake
